<a href="https://colab.research.google.com/github/peisuke/MagicAnimateHandson/blob/main/03_openpose_for_magic_animate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 最初に、ランタイムの種類を「GPU」に変更してください。（ランタイム＞ランタイムのタイプを変更）
# GPUが有効になっていることを確認します。

# First, change the runtime type to "GPU".
# Confirm runtime to GPU
! nvcc --version
! nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
Wed Dec  6 14:04:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |       

In [ ]:
!wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
!tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

In [ ]:
!git clone --depth 1 -b v1.7.0 https://github.com/CMU-Perceptual-Computing-Lab/openpose.git
%cd openpose

Cloning into 'openpose'...
remote: Enumerating objects: 659, done.
remote: Counting objects: 100% (659/659), done.
remote: Compressing objects: 100% (625/625), done.
remote: Total 659 (delta 171), reused 162 (delta 24), pack-reused 0
Receiving objects: 100% (659/659), 46.51 MiB | 36.75 MiB/s, done.
Resolving deltas: 100% (171/171), done.
Note: switching to '8ca5c1d95a42340b323e9273654d1db98bec779c'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

/content/openpose


In [ ]:
# Basic
!sudo apt-get --assume-yes update
!sudo apt-get --assume-yes install build-essential

# OpenCV
!sudo apt-get --assume-yes install libopencv-dev

# General dependencies
!sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
!sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev

# Remaining dependencies, 14.04
!sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev

# Python3 libs
!sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
!sudo apt-get --assume-yes install python3-pip
!sudo -H pip3 install --upgrade numpy protobuf opencv-python

# OpenCL Generic
!sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
!sudo apt-get --assume-yes install libviennacl-dev

In [ ]:
# copy open pose model from https://drive.google.com/drive/folders/1USEdy_7uvwO4PIqsQJq8kT0sX4H4f7nn?usp=drive_link
!cp "/content/drive/MyDrive/AI/video/openpose/pose_iter_584000.caffemodel" models/pose/body_25/
!cp "/content/drive/MyDrive/AI/video/openpose/pose_iter_102000.caffemodel" models/hand/
!cp "/content/drive/MyDrive/AI/video/openpose/pose_iter_116000.caffemodel" models/face/

In [ ]:
!rm -rf build && mkdir build && cd build && cmake -DUSE_CUDNN=OFF .. && make -j`nproc`

In [ ]:
!./build/examples/openpose/openpose.bin \
    --video "video.mp4" \
    --display 0 \
    --write_video "bone.mp4" \
    --face --disable_blending

In [21]:
import cv2
import numpy as np

cap = cv2.VideoCapture("bone.mp4")

img_width = 512
img_height = 512
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter("bone2.mp4",
                         fourcc, 25.0, (img_width, img_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img = frame
    # imgの横にpaddingを入れる
    pad = (img.shape[0] - img.shape[1]) // 2
    tmp = np.pad(img, ((0, 0), (pad, pad), (0, 0)), 'edge')
    tmp = cv2.resize(tmp, (img_width, img_height))
    writer.write(tmp)
writer.release()